# A3 - Neural Machine Translation (Myanmar to English)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext, datasets, math
from tqdm import tqdm # progress bar
import pickle

## 1. Load Data

In [4]:
dataset = datasets.load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 18088
    })
    validation: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
        num_rows: 1019
    })
})

In [6]:
dataset['train']

Dataset({
    features: ['SNT.URLID', 'SNT.URLID.SNTID', 'url', 'translation'],
    num_rows: 18088
})

In [7]:
dataset['train'][12]

{'SNT.URLID': '87564',
 'SNT.URLID.SNTID': '13',
 'url': 'http://en.wikinews.org/wiki/Data_for_3_million_UK_driving_candidates_lost',
 'translation': {'bg': 'অক্টোবরে এইচএম রেভিনিউ ২৫ মিলিয়ন লোকের তথ্য হারিয়ে ফেলার পরে এটিই হল ইউকে-তে প্রথম এত বড় তথ্যের ক্ষতি।',
  'en': 'It is the first major loss of data in the UK since information on 25 million people was lost by HM Revenue in October.',
  'en_tok': 'It is the first major loss of data in the UK since information on 25 million people was lost by HM Revenue in October .',
  'fil': 'Ito ang unang malakihang pagkawala ng data sa UK dahil ang impormasyon sa 25 milyong tao ay nawalan ng HM Revenue noong Oktubre.',
  'hi': 'यह ब्रिटेन में डेटा का पहला बड़ा नुकसान है क्योंकि अक्टूबर में HM रेवेन्यू द्वारा 25 मिलियन लोगों की जानकारी गुम हो गई थी।',
  'id': 'Ini adalah kehilangan data yang besar pertama di UK sejak hilangnya informasi tentang 25 juta orang oleh HM Revenue di bulan Oktober.',
  'ja': 'これは、10月に歳入関税庁が2500万人分の情報を失って以来初めてのイギリスでの大きな

In [8]:
dataset['train'][11]['translation']

{'bg': 'গতকাল ১৭৩০ ইউটিসি-তে হাউস অফ্\u200c কমন্\u200cস্\u200c-এ ইউকে-র পরিবহন সচিব রুথ কেলি এই তথ্যগুলি দিয়েছেন।',
 'en': 'Details were given by the UK Transport Secretary, Ruth Kelly, in the House of Commons at 1730 UTC yesterday.',
 'en_tok': 'Details were given by the UK Transport Secretary , Ruth Kelly , in the House of Commons at 1730 UTC yesterday .',
 'fil': 'Ang mga detalye ay ibinigay ng UK transport Secretary, na si Ruth Kelly, sa House of Commons sa ika-17:30 UTC kahapon.',
 'hi': 'कल ब्रिटेन के परिवहन सचिव रूथ केली द्वारा 1730 UTC पर हाउस ऑफ़ कॉमन्स में विवरण दिए गए।',
 'id': 'Detil diberikan oleh Sekretaris Kementerian Transportasi UK, Ruth Kelly, di Dewan Perwakilan Rakyat kemarin 17:30 UTC.',
 'ja': '詳細は昨日UTC17時30分、英国議会でイギリスのルス・ケリー運輸大臣によって伝えられた。',
 'khm': 'ព័ត៌មានលំអិតត្រូវបានផ្តល់ដោយរដ្ឋមន្ត្រីដឹកជញ្ចូន លោករ៉ូថ ខេលលី នៅក្នុងសភានៅម៉ោង1730ម្សិលមិញ។',
 'lo': 'ຂໍ້ມູນໄດ້ຖືກສະໜອງໂດຍ ເລຂາທິການຂົນສົ່ງ ສະຫະລາຊະອານາຈັກ ຣູດ ເຄລີ່ ໃນສະພາຕໍ່າ ທີ່ 1730 UTC ມື້ວານນີ້.',
 'ms': 'Butir

In [9]:
dataset['train'][11]['translation']['en']

'Details were given by the UK Transport Secretary, Ruth Kelly, in the House of Commons at 1730 UTC yesterday.'

In [10]:
dataset['train'][11]['translation']['my']

'အသေးစိတ်များ ကို မနေ့က ၁၇၃၀ ယူတီစီ ၌ အောက်လွှတ်တော် ရှိ ဗြိတိန်နိုင်ငံ ပို့ဆောင်ရေး အတွင်းရေးမှူး ရုသ်ကယ်လီ က ပေးခဲ့သည် ။'

In [13]:
languages = ['my', 'en']

# english myanmar data
datasetENMY_train = [{lang: row['translation'][lang] for lang in languages} for row in dataset['train']]

In [18]:
len(datasetENMY_train)

18088

In [17]:
trg = datasetENMY_train[0]['my']
src = datasetENMY_train[1]['en']

In [19]:
source = [] # for english language
target = [] # for myanmar language

for i in range (len(datasetENMY_train)):
    source.append(datasetENMY_train[i]['en'])
    target.append(datasetENMY_train[i]['my'])

In [20]:
source[1]

'Andrea Masi opened the scoring in the fourth minute with a try for Italy.'

In [21]:
target[1]

'အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု အဖြစ် စတုတ္ထ မိနစ် တွင် အမှတ်ပေးခြင်း ကို ဖွင့်လှစ်ပေးခဲ့သည် ။'